In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px

In [ ]:
df = pd.read_csv('/content/Przejazd_2.csv', sep=';')
df

,T1,T2,P1,P2
0,6799,6765,32899,32636
1,6797,6764,32899,32636
2,6794,6760,32900,32636
3,6793,6761,32900,32637
4,6797,6764,32898,32636
...,...,...,...,...
7885,6799,6766,32900,32636
7886,6788,6759,32899,32635
7887,6798,6760,32897,32638
7888,6793,6765,32899,32638


In [ ]:
px.line(df, y=['T1', 'T2'])

### Select the areas and get local maxima

In [ ]:
slice_t2_front_left_wheel = df.iloc[44:115, 1]
slice_t1_front_right_wheel = df.iloc[576:660, 0]
slice_t2_rear_left_wheel = df.iloc[1222:1296, 1]
slice_t1_rear_right_wheel = df.iloc[1707:1767, 0]

In [ ]:
t2_front_left_max_index = slice_t2_front_left_wheel[
    slice_t2_front_left_wheel == slice_t2_front_left_wheel.max()].index
t1_front_right_max_index = slice_t1_front_right_wheel[
    slice_t1_front_right_wheel == slice_t1_front_right_wheel.max()].index

t2_rear_left_max_index = slice_t2_rear_left_wheel[
    slice_t2_rear_left_wheel == slice_t2_rear_left_wheel.max()].index
t1_rear_right_max_index = slice_t1_rear_right_wheel[
    slice_t1_rear_right_wheel == slice_t1_rear_right_wheel.max()].index

In [ ]:
t2_front_left_max_index, t1_front_right_max_index, t2_rear_left_max_index, t1_rear_right_max_index

(Int64Index([76], dtype='int64'),
 Int64Index([615], dtype='int64'),
 Int64Index([1253], dtype='int64'),
 Int64Index([1733], dtype='int64'))

### Calculation the axles speeds

In [ ]:
DISTANCE_BETWEEN_SENSORS = 2

front_axle_speed = (DISTANCE_BETWEEN_SENSORS 
                    / ((t1_front_right_max_index - t2_front_left_max_index) 
                    / 1500.))[0]

rear_axle_speed = (DISTANCE_BETWEEN_SENSORS 
                   / ((t1_rear_right_max_index - t2_rear_left_max_index) 
                   / 1500.))[0]

avg_axle_speed = np.mean([front_axle_speed, rear_axle_speed])

acceleration = ((rear_axle_speed - front_axle_speed) 
                 / ((t1_front_right_max_index - t2_front_left_max_index) 
                 / 1500.))[0]


front_axle_speed, rear_axle_speed, avg_axle_speed, acceleration

(5.565862708719852, 6.25, 5.907931354359926, 1.903907118590394)

In [ ]:
SENSOR_READING_PER_SECOND = 1500.

distance_between_left_side_wheels = (
    ((t2_rear_left_max_index - t2_front_left_max_index)
     / SENSOR_READING_PER_SECOND) * avg_axle_speed)[0]

distance_between_right_side_wheels = (
    ((t1_rear_right_max_index - t1_front_right_max_index) 
    / SENSOR_READING_PER_SECOND) * avg_axle_speed)[0]

distance_between_left_side_wheels, distance_between_right_side_wheels

(4.6357568027210885, 4.4033781694495975)

### Calculate the areas under the charts

In [ ]:
def integral(area):
  # Reduce the area by a constant sensor reading: 6800
  area -= 6800
  return np.trapz(area, dx=1)

area_under_chart_front_left_wheel = integral(
    area=slice_t2_front_left_wheel.copy())
area_under_chart_front_right_wheel = integral(
    area=slice_t1_front_right_wheel.copy())
area_under_chart_rear_left_wheel = integral(
    area=slice_t2_rear_left_wheel.copy())
area_under_chart_rear_right_wheel = integral(
    area=slice_t1_rear_right_wheel.copy())

(area_under_chart_front_left_wheel, area_under_chart_front_right_wheel, 
area_under_chart_rear_left_wheel, area_under_chart_rear_right_wheel)

(128299.5, 85759.5, 100138.0, 80276.0)

### Calculate axle pressures and loads

In [ ]:
WEIGHT_OF_FRONT_AXLE = 1400

C = WEIGHT_OF_FRONT_AXLE / ((area_under_chart_front_left_wheel*front_axle_speed 
                             + area_under_chart_front_right_wheel*front_axle_speed) 
  * front_axle_speed)

WEIGHT_OF_REAR_AXLE = ((area_under_chart_rear_left_wheel*rear_axle_speed 
                       + area_under_chart_rear_right_wheel*rear_axle_speed) 
  * C * rear_axle_speed)

C, WEIGHT_OF_REAR_AXLE

(0.0002111200909821851, 1487.8523474398414)

In [ ]:
percentage_weight_left_axle = (area_under_chart_front_left_wheel 
                               / (area_under_chart_front_left_wheel 
                                  + area_under_chart_front_right_wheel))
percentage_weight_right_axle = (area_under_chart_front_right_wheel 
                                / (area_under_chart_front_right_wheel 
                                   + area_under_chart_front_left_wheel))

percentage_weight_left_axle, percentage_weight_right_axle

(0.5993651283057475, 0.4006348716942525)

In [ ]:
pressure_front_left_wheel = percentage_weight_left_axle * WEIGHT_OF_FRONT_AXLE
pressure_front_right_wheel = percentage_weight_right_axle * WEIGHT_OF_FRONT_AXLE

pressure_front_left_wheel, pressure_front_right_wheel

(839.1111796280466, 560.8888203719534)

In [ ]:
c_left_wheels = (pressure_front_left_wheel 
                 / ((area_under_chart_front_left_wheel * front_axle_speed) 
                 * front_axle_speed))
c_right_wheels = (pressure_front_right_wheel 
                  / ((area_under_chart_front_right_wheel * front_axle_speed) 
                  * front_axle_speed))

c_left_wheels, c_right_wheels

(0.00021112009098218508, 0.00021112009098218503)

In [ ]:
pressure_rear_left_wheel = (c_left_wheels * (area_under_chart_rear_left_wheel 
                                             * rear_axle_speed) 
                            * rear_axle_speed)
pressure_rear_right_wheel = (c_right_wheels * (area_under_chart_rear_right_wheel 
                                               * rear_axle_speed) 
                             * rear_axle_speed)

pressure_rear_left_wheel, pressure_rear_right_wheel

(825.8259246396113, 662.0264228002299)

### Calculate distances between piezoelectric and strain gauge sensors

In [ ]:
df.head()

,T1,T2,P1,P2
0,6799,6765,32899,32636
1,6797,6764,32899,32636
2,6794,6760,32900,32636
3,6793,6761,32900,32637
4,6797,6764,32898,32636


In [ ]:
px.line(df, y=['P1', 'P2'])

In [ ]:
slice_p2_front_left_wheel = df.iloc[1972:2040, 3]
slice_p1_front_right_wheel = df.iloc[1933:1995, 2]
slice_p2_rear_left_wheel = df.iloc[3000:3050, 3]
slice_p1_rear_right_wheel = df.iloc[3006:3050, 2]

p2_front_left_max_index = slice_p2_front_left_wheel[
  slice_p2_front_left_wheel == slice_p2_front_left_wheel.max()].index[0]
p1_front_right_max_index = slice_p1_front_right_wheel[
  slice_p1_front_right_wheel == slice_p1_front_right_wheel.max()].index[0]
p2_rear_left_max_index = slice_p2_rear_left_wheel[
  slice_p2_rear_left_wheel == slice_p2_rear_left_wheel.max()].index[0]
p1_rear_right_max_index = slice_p1_rear_right_wheel[
  slice_p1_rear_right_wheel == slice_p1_rear_right_wheel.max()].index[0]

(p2_front_left_max_index, p1_front_right_max_index, 
 p2_rear_left_max_index, p1_rear_right_max_index)

(2003, 1967, 3029, 3024)

In [ ]:
time_between_sensors_left_front_wheel = ((p2_front_left_max_index 
                                          - t2_front_left_max_index[0]) 
                                         / SENSOR_READING_PER_SECOND)

time_between_sensors_right_front_wheel = ((p1_front_right_max_index 
                                          - t1_front_right_max_index[0]) 
                                         / SENSOR_READING_PER_SECOND)

time_between_sensors_left_rear_wheel = ((p2_rear_left_max_index 
                                          - t2_rear_left_max_index[0]) 
                                         / SENSOR_READING_PER_SECOND)

time_between_sensors_right_rear_wheel = ((p1_rear_right_max_index 
                                          - t1_rear_right_max_index[0]) 
                                         / SENSOR_READING_PER_SECOND)

(time_between_sensors_left_front_wheel, time_between_sensors_right_front_wheel, 
 time_between_sensors_left_rear_wheel, time_between_sensors_right_rear_wheel)

(1.2846666666666666, 0.9013333333333333, 1.184, 0.8606666666666667)

In [ ]:
distance_between_left_side_sensors = front_axle_speed * time_between_sensors_left_front_wheel
distance_between_right_side_sensors = front_axle_speed * time_between_sensors_right_front_wheel

distance_between_left_side_sensors, distance_between_right_side_sensors

(7.150278293135436, 5.01669758812616)